<a target="_blank" href="https://colab.research.google.com/github/avakanski/Graph_NNs_for_Multi_Omics_Integration/blob/main/Code/GAT_MultiOmics_CorrMat.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.9 MB/s eta 0:00:00


In [ ]:
# GAT using multi-omics data (mRNA, miRNA and DNA methylation) with correlation matrix graph structure (5 fold cross validation)
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from torch.nn import BatchNorm1d, LeakyReLU
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import datetime
now = datetime.datetime.now

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Step 1: Load the PPI data
ppi_file_path = 'drive/My Drive/Projects/Gene_Expression_Project/PPI.csv'
ppi_df = pd.read_csv(ppi_file_path)

# Step 2: Concatenate 'stringId_A' and 'stringId_B' to calculate the number of connections (degree)
all_proteins = pd.concat([ppi_df['stringId_A'], ppi_df['stringId_B']])

# Step 3: Count the number of connections for each protein
protein_connections = all_proteins.value_counts()

# Step 4: Define a degree threshold to select only highly connected proteins (e.g., 200 or more connections)
degree_threshold = 200
high_degree_proteins = protein_connections[protein_connections >= degree_threshold].index

# Step 5: Filter the PPI data to include only edges where both proteins have a high number of connections
ppi_filtered = ppi_df[ppi_df['stringId_A'].isin(high_degree_proteins) & ppi_df['stringId_B'].isin(high_degree_proteins)]

# Step 6: Map the high-degree proteins to unique node IDs
proteins = pd.concat([ppi_filtered['stringId_A'], ppi_filtered['stringId_B']]).unique()
protein_to_id = {protein: idx for idx, protein in enumerate(proteins)}

# Step 7: Create edge index (this will be the input for GAT)
edges = ppi_filtered[['stringId_A', 'stringId_B']].applymap(lambda x: protein_to_id[x])
edge_index = torch.tensor(edges.values.T, dtype=torch.long).to(device)  # Move edge index to device

<ipython-input-4-dbafd6daffc1>:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  edges = ppi_filtered[['stringId_A', 'stringId_B']].applymap(lambda x: protein_to_id[x])


In [ ]:
# Step 8: Load and preprocess the multi-omics data
!wget https://www.webpages.uidaho.edu/vakanski/Codes_Data/mRNA_miRNA_Meth_integrated.csv
file_path = 'mRNA_miRNA_Meth_integrated.csv'
df = pd.read_csv(file_path)
df.drop(df.columns[0], axis=1, inplace=True)
Y = df.iloc[:, -1].copy()

# Remove non-numeric columns
df = df.select_dtypes(include=[np.number])
X = df.values

num_classes = len(set(Y))
print("Number of classes:", num_classes)
num_samples = X.shape[0]
print("Number of samples:", num_samples)
num_Features = X.shape[1]
print("Number of Features:", num_Features)

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

--2025-03-07 19:10:57--  https://www.webpages.uidaho.edu/vakanski/Codes_Data/mRNA_miRNA_Meth_integrated.csv
Resolving www.webpages.uidaho.edu (www.webpages.uidaho.edu)... 129.101.105.230
Connecting to www.webpages.uidaho.edu (www.webpages.uidaho.edu)|129.101.105.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123599052 (118M) [application/octet-stream]
Saving to: ‘mRNA_miRNA_Meth_integrated.csv’

mRNA_miRNA_Meth_int 100%[===================>] 117.87M  17.0MB/s    in 9.5s    

2025-03-07 19:11:08 (12.4 MB/s) - ‘mRNA_miRNA_Meth_integrated.csv’ saved [123599052/123599052]

Number of classes: 32
Number of samples: 8464
Number of Features: 2793


In [ ]:
# Step 9: Define the GAT model
class GAT(nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.conv1 = GATConv(X.shape[1], 1024, heads=8)
        self.bn1 = BatchNorm1d(1024 * 8)
        self.relu1 = LeakyReLU()
        self.conv2 = GATConv(1024 * 8, 512, heads=4)
        self.bn2 = BatchNorm1d(512 * 4)
        self.relu2 = LeakyReLU()
        self.conv3 = GATConv(512 * 4, 256, heads=2)
        self.bn3 = BatchNorm1d(256 * 2)
        self.relu3 = LeakyReLU()
        self.conv4 = GATConv(256 * 2, 32, heads=1)
        self.bn4 = BatchNorm1d(32)
        self.relu4 = LeakyReLU()
        self.dropout = nn.Dropout(0.3)  # Apply the specified dropout rate

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.dropout(x)
        x = self.conv4(x, edge_index)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.dropout(x)
        return F.log_softmax(x, dim=1)

# Step 10: Set up K-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True)

# Initialize lists to store metrics for each fold
precision_scores = []
recall_scores = []
accuracy_scores = []
F1Measure = []

In [ ]:
# Step 11: Training and Evaluation
t = now()
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    X_train = torch.FloatTensor(X_train).to(device)
    y_train = torch.LongTensor(y_train).to(device)
    X_test = torch.FloatTensor(X_test).to(device)
    y_test = torch.LongTensor(y_test).to(device)

    # Calculate the correlation matrix and convert it to an edge index
    correlation_matrix_train = np.corrcoef(X_train.cpu(), rowvar=True)
    correlation_matrix_test = np.corrcoef(X_test.cpu(), rowvar=True)

    # Create edge indices based on a correlation threshold
    src_train, dst_train = np.where(correlation_matrix_train > 0.9)
    src_test, dst_test = np.where(correlation_matrix_test > 0.9)

    edge_index_train = torch.tensor([src_train, dst_train], dtype=torch.long).to(device)
    edge_index_test = torch.tensor([src_test, dst_test], dtype=torch.long).to(device)

    # Create PyTorch Geometric data objects
    train_data = Data(x=X_train, edge_index=edge_index_train).to(device)
    test_data = Data(x=X_test, edge_index=edge_index_test).to(device)

    # Create the GAT model with tuned parameters
    model = GAT().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)  # Set the specified learning rate
    criterion = nn.NLLLoss()

    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        out = model(train_data)
        loss = criterion(out, y_train)
        loss.backward()
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            logits = model(test_data)
            pred = torch.argmax(logits, dim=1)
            acc = accuracy_score(y_test.cpu().numpy(), pred.cpu().numpy())
            # print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Accuracy: {acc:.4f}')

    # Testing
    model.eval()
    with torch.no_grad():
        logits = model(test_data)
        pred = torch.argmax(logits, dim=1)
        test_acc = accuracy_score(y_test.cpu().numpy(), pred.cpu().numpy())
        precision = precision_score(y_test.cpu().numpy(), pred.cpu().numpy(), average='macro', zero_division=1)
        recall = recall_score(y_test.cpu().numpy(), pred.cpu().numpy(), average='macro')
        f1 = f1_score(y_test.cpu().numpy(), pred.cpu().numpy(), average='macro')

        accuracy_scores.append(test_acc)
        precision_scores.append(precision)
        recall_scores.append(recall)
        F1Measure.append(f1)
print('Training time: %s' % (now() - t))

<ipython-input-7-ce55b38050fd>:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  edge_index_train = torch.tensor([src_train, dst_train], dtype=torch.long).to(device)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Training time: 0:04:19.037287


In [ ]:
# Calculate the average metrics across all folds
average_accuracy = np.mean(accuracy_scores)
average_precision = np.mean(precision_scores)
average_recall = np.mean(recall_scores)
average_f1 = np.mean(F1Measure)

print("Average accuracy =", average_accuracy)
print("Accuracy std sev =", np.std(accuracy_scores))
print("Average precision =", average_precision)
print("Precision std sev =", np.std(precision_scores))
print("Average recall =", average_recall)
print("Recall std sev =", np.std(recall_scores))
print("Average F1 score =", average_f1)
print("F1 std dev =", np.std(F1Measure))

Average accuracy = 0.958530327213013
Accuracy std sev = 0.0024311323943563123
Average precision = 0.9333677756849544
Precision std sev = 0.020768593440016647
Average recall = 0.9364388490361402
Recall std sev = 0.015947474761482523
Average F1 score = 0.9335732312352951
F1 std dev = 0.01782216803131316
